In [3]:
!pip install --upgrade pip
!pip install --upgrade transformers datasets torch scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Using cached datasets-3.4.1-py3-none-any.whl.metadata (19 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 121.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 175.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 176.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 68.6 MB/s eta 

In [41]:
!pip install evaluate

In [52]:
!pip install wandb

In [42]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from datasets import load_dataset
import evaluate
import torch
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


In [29]:
df = pd.read_csv('/content/train.csv')
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [33]:
df["toxic_label"] = df[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].max(axis=1)
df = df[["comment_text", "toxic_label"]]


In [34]:
# Convert to Hugging Face dataset
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['comment_text', 'toxic_label'],
    num_rows: 159571
})

In [35]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize function
def tokenize_function(examples):
    return tokenizer(examples["comment_text"], padding="max_length", truncation=True)

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove original text column
tokenized_datasets = tokenized_datasets.remove_columns(["comment_text"])
tokenized_datasets = tokenized_datasets.rename_column("toxic_label", "labels")
tokenized_datasets.set_format("torch")

# Split into train and validation
train_dataset = tokenized_datasets.train_test_split(test_size=0.1)["train"]
val_dataset = tokenized_datasets.train_test_split(test_size=0.1)["test"]

print("Dataset is ready!")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/159571 [00:00<?, ? examples/s]

Dataset is ready!


In [37]:
# Load model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [64]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.001,
    logging_dir="./logs",
    logging_steps=1000,
    load_best_model_at_end=True,
    fp16=True,
    report_to="none",
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [70]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)

    # Calculate precision within compute_metrics
    precision = evaluate.load("precision").compute(predictions=predictions, references=labels)

    accuracy = metric.compute(predictions=predictions, references=labels)
    recall = evaluate.load("recall").compute(predictions=predictions, references=labels)
    f1 = evaluate.load("f1").compute(predictions=predictions, references=labels)
    return {"accuracy": accuracy["accuracy"], "precision": precision["precision"], "recall": recall["recall"], "f1": f1["f1"]}

In [53]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [71]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

<ipython-input-71-d411c8a07fbe>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.090800,0.062238,0.976501,0.893901,0.871207,0.882408


TrainOutput(global_step=8976, training_loss=0.09819510318803702, metrics={'train_runtime': 1861.4499, 'train_samples_per_second': 77.151, 'train_steps_per_second': 4.822, 'total_flos': 1.902404052318413e+16, 'train_loss': 0.09819510318803702, 'epoch': 1.0})

In [72]:
test_results = trainer.evaluate(val_dataset)
print("Evaluation Results:", test_results)

Evaluation Results: {'eval_loss': 0.06223754584789276, 'eval_accuracy': 0.9765008146384259, 'eval_precision': 0.8939008894536213, 'eval_recall': 0.8712074303405573, 'eval_f1': 0.8824082784571966, 'eval_runtime': 63.9415, 'eval_samples_per_second': 249.572, 'eval_steps_per_second': 15.608, 'epoch': 1.0}


In [75]:
# Inference function with proper device handling
def classify_text(text):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Ensure the model is on the correct device

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)  # Move inputs to the same device
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1).item()
    return "Unsafe" if prediction == 1 else "Safe"


In [76]:
# Test with some examples
test_texts = [
    "I appreciate your help, thank you!",
    "You are so stupid and annoying!",
]

for text in test_texts:
    print(f"Text: {text} -> Classification: {classify_text(text)}")

Text: I appreciate your help, thank you! -> Classification: Safe
Text: You are so stupid and annoying! -> Classification: Unsafe


# Documentation & Deployment
# Readme/Notebook Documentation:
# - Approach: Fine-tuned DistilBERT for binary classification of safe vs. unsafe text.
# - Hyperparameters: Optimized for Colab with reduced batch size and minimal epochs.
# - Trade-offs: Limited training time vs. performance trade-off.

# Potential Extensions:
# - Could be deployed in a real-time system using a FastAPI or Flask API.
# - Integrated with moderation tools to filter out unsafe content before generation.
# - Further fine-tuning on a larger dataset could improve performance in real-world applications.